In [8]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from os import listdir
import numpy as np
import math
from scipy.spatial import ConvexHull
from scipy.spatial.distance import cdist
from scipy.spatial import distance
from itertools import groupby
import json

In [2]:
def calculateMass(data):
#      multiply by constant 917
    result = 0
    for d in data:
        result += (1/6)*math.pi*math.pow(d,3) * 917 * math.pow(10, 6)
#     return round(result, 3)
    return result

In [3]:
def calculateLength(data):
    """
    find the most distant point
    data = a nupmpy array containing the points
    """
    """first check if the x points are same or not
    if same we will use the naive techniques
    if not we will use the convex hull approach
    """
    xVals = [i[0] for i in data]
#     print(xVals)
    g = groupby(xVals)
    sameValues = next(g, True) and not next(g, False)
#     print(sameValues)
    
    if sameValues == False:
        # # Find a convex hull in O(N log N)
        hull = ConvexHull(data)

        # Extract the points forming the hull
        hullpoints = data[hull.vertices,:]
    
    elif sameValues == True:
        hullpoints = data       

#     print(hullpoints)
    
    # Naive way of finding the best pair in O(H^2) time if H is number of points on
    # hull
    hdist = cdist(hullpoints, hullpoints, metric='euclidean')

    # Get the farthest apart points
    bestpair = np.unravel_index(hdist.argmax(), hdist.shape)

    #Print them
    points = [hullpoints[bestpair[0]],hullpoints[bestpair[1]]]
#     print(points)

    # print the distance
#     dist = np.linalg.norm(points[0] - points[1])
    dst = distance.euclidean(points[0], points[1])
#     print(points[0], points[1], dst)
    return round(dst, 3)

In [9]:
columns = ["cluster", "nodeCluster", "temperature", "ice_particles", "ice_mass", "ice_length"]
folders = [f for f in listdir("data/withOutliers/")]
# folders = ['newData']
for folder in folders:
    onlyfiles = [f for f in listdir("data/withOutliers/{}".format(folder)) ]
    simulationData = {}
#     print(folder)
    for file in onlyfiles:
        path = "data/withOutliers/{}/{}".format(folder, file)
#         print(path)
        df = pd.read_csv(path)
#             calculating the diameter in micron (?)
#         df['new_d'] = df['d'] * math.pow(10, 6)
    
#         getting the temperature means
        means = round(df.groupby('cluster')['T'].mean(), 3)
#         print(df.groupby('cluster')['nodeCluster'])
#     getting the clusters
        clusters = means.index.tolist()
#         print(clusters)
#     getting the nodeCluster
        nodeClusters = sorted(df['nodeCluster'].value_counts().index.tolist())
#         print(nodeClusters)
#     storing the temperature
        temperature = means.tolist()
#     storing total particles
        particles = df.groupby('cluster')['cluster'].value_counts().tolist()

#     adding the total values for all data
        clusters.append("all")
        nodeClusters.append('all')
        temperature.append(round(df['T'].mean(), 3))
        particles.append(df.shape[0])

#         extracting the total volumes
#         volumes = []
        volumesExact = []
#         extracting the farthest distance
        distances = []
        for each in clusters:
#             print(each)
            if each == 'all':
#                 print(each)
#                 volumes.append(calculateVolume(df['new_d'].tolist()))
                volumesExact.append(calculateMass(df['d'].tolist()))
                distances.append(calculateLength(df[['Points:0', 'Points:1', 'Points:2']].to_numpy()))
            else:
#                 print(each)
#                 volumes.append(calculateVolume(df[df.cluster == each]['new_d'].tolist()))
                volumesExact.append(calculateMass(df[df.cluster == each]['d'].tolist()))
                distances.append(calculateLength(df[df.cluster == each][['Points:0', 'Points:1', 'Points:2']].to_numpy()))
        
        
        # print(means)
#         print(clusters)
#         print(temperature)
#         print(particles)
#         print(volumes)
#         print(distances)
#         print(volumesExact)
        for index, value in enumerate(nodeClusters):
            if value == 'all':
                pass
            else:
                simulationData[value] = {'temp': temperature[index], "mass": volumesExact[index], 'length': distances[index]}
            
    print(simulationData)
    with open('output/jsons/{}-attr.json'.format(folder), 'w', encoding='utf-8') as outfile:
        json.dump(simulationData, outfile, sort_keys=True, indent=4)
# output csv generation
#         extract = pd.DataFrame(np.column_stack([clusters, nodeClusters, temperature, particles, volumesExact, distances]),
#                       columns=columns)

#         extract.to_csv('output/withOutliers/{}/{}'.format(folder, file), index=False)
#         print("exported {} {} ".format(folder, file))


{1: {'temp': 222.646, 'mass': 0.0015629643429727138, 'length': 13.088}, 2: {'temp': 232.98, 'mass': 0.025341647172142007, 'length': 10.196}, 3: {'temp': 265.783, 'mass': 0.0013469086708670832, 'length': 1.569}, 4: {'temp': 266.405, 'mass': 0.00015606126483606192, 'length': 0.359}, 5: {'temp': 303.108, 'mass': 9.5985905776905e-05, 'length': 0.237}, 6: {'temp': 272.61, 'mass': 6.16057629163639e-05, 'length': 0.203}, 7: {'temp': 219.009, 'mass': 0.001217981487420209, 'length': 7.426}, 8: {'temp': 233.131, 'mass': 0.028818638212496635, 'length': 11.851}, 9: {'temp': 359.682, 'mass': 0.0001020254794922879, 'length': 0.345}, 10: {'temp': 306.275, 'mass': 8.334937646470519e-05, 'length': 0.269}, 11: {'temp': 324.15, 'mass': 3.772382125003906e-05, 'length': 0.155}, 12: {'temp': 337.77, 'mass': 1.8792591401373944e-05, 'length': 0.129}, 13: {'temp': 219.171, 'mass': 0.0007095402511161619, 'length': 8.686}, 14: {'temp': 234.678, 'mass': 0.029893554646748487, 'length': 13.182}, 15: {'temp': 219.12

{1: {'temp': 216.42, 'mass': 1.6292174499227848e-07, 'length': 46.582}, 2: {'temp': 234.971, 'mass': 2.168578663355183e-05, 'length': 23.152}, 3: {'temp': 231.477, 'mass': 1.4223582785705508e-07, 'length': 4.086}, 4: {'temp': 236.224, 'mass': 7.045248161459013e-08, 'length': 1.601}, 5: {'temp': 232.375, 'mass': 2.6644169915598147e-09, 'length': 1.664}, 6: {'temp': 236.302, 'mass': 1.1210202378404518e-11, 'length': 1.295}, 7: {'temp': 219.122, 'mass': 2.7014515011987703e-07, 'length': 37.59}, 8: {'temp': 233.861, 'mass': 1.8017285942319854e-05, 'length': 24.829}, 9: {'temp': 234.211, 'mass': 2.1798409345561837e-08, 'length': 1.901}, 10: {'temp': 233.577, 'mass': 1.6301858620381483e-09, 'length': 1.338}, 11: {'temp': 244.8, 'mass': 1.383133485693005e-10, 'length': 1.11}, 12: {'temp': 233.212, 'mass': 3.6092235133791862e-12, 'length': 1.11}, 13: {'temp': 233.385, 'mass': 1.069072752985278e-10, 'length': 1.138}, 14: {'temp': 219.19, 'mass': 7.370019650735017e-08, 'length': 37.059}, 15: {'t

In [10]:
path = "data/withoutOutliers/{}/{}".format("contrails1", "0.2.csv")
df = pd.read_csv(path)
pts = df[['Points:0', 'Points:1', 'Points:2']].to_numpy()
hull = ConvexHull(p)

In [14]:

from mpl_toolkits.mplot3d import Axes3D


# 8 points defining the cube corners
# pts = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0],
#                 [0, 0, 1], [1, 0, 1], [1, 1, 1], [0, 1, 1], ])

hull = ConvexHull(pts)

# fig = plt.figure()
# ax = fig.add_subplot(111, projection="3d")

# # Plot defining corner points
# ax.plot(pts.T[0], pts.T[1], pts.T[2], "ko")

# # 12 = 2 * 6 faces are the simplices (2 simplices per square face)
# for s in hull.simplices:
#     s = np.append(s, s[0])  # Here we cycle back to the first coordinate
#     ax.plot(pts[s, 0], pts[s, 1], pts[s, 2], "r-")

# # Make axis label
# for i in ["x", "y", "z"]:
#     eval("ax.set_{:s}label('{:s}')".format(i, i))

# plt.show()

print(hull)

In [7]:
x = rand(1,20);              # 20 random x values
y = rand(1,20);              # 20 random y values
hullPoints = convhull(x,y);  # Find the points defining the convex hull
patch(x(hullPoints),y(hullPoints),'r');  # Plot the convex hull in red
hold on;                     # Add to the existing plot
scatter(x,y);    

SyntaxError: invalid syntax (Temp/ipykernel_7936/3301890463.py, line 5)

In [8]:
[4.8062e+00 3.8889e-01 5.9698e-12] [ 5.29590000e+01 -1.19520000e+00 -4.06809985e-09] 48.17884897927824

SyntaxError: invalid syntax (Temp/ipykernel_7936/1113393829.py, line 1)

In [11]:
5.29590000e+01

52.959